In [ ]:
# После выполнения этой ячейки нужно перезапустить среду выполнения

!pip uninstall scikit-learn
!pip install scikit-learn==0.23.2
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText
!sudo pip install fastText
!pip install compress-fasttext[full]

# Обработка данных

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('prepared_data.csv') #загружаем датасет

In [ ]:
pd.show_versions()

In [ ]:
#стоп-слова

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('russian')
for w in ['не', 'для', 'при', 'как', 'от', 'на', 'из', 'за', 'yt', 'fc', '.']:
    stop.append(w)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#словарь аббревиатур из датасета

abbr_dict = {'сигма': 'сеть компьютер',
             'фпсу': 'фильтр пакет сетевой уровень',
             'оптикэш': 'оптимизация заявка перемещение денежный средства внутренний структурный подразделение',
             'opticash': 'оптимизация заявка перемещение денежный средства внутренний структурный подразделение',
             'вко': 'владелец карточка организация',
             'альфа': 'сеть',
             'ппрб': 'платформа поддержка развитие бизнес',
             'alpha': 'сеть',
             'всп': 'внутренний структурный подразделение',
             'citrix': 'виртуальный автоматический рабочий место',
             'туз': 'технический учетный запись',
             'цитрикс': 'виртуальный автоматический рабочий место',
             'екс': 'единый корпоративный система',
             'вкс': 'владелец карточка сделка',
             'впн': 'сеть', 'vpn': 'сеть',
             'арм': 'автоматизированный рабочий место',
             'варм': 'виртуальный автоматизированный рабочий место',
             'егрн': 'единый государственный реестр недвижимость',
             'ас': 'автоматизированный система',
             'сбо': 'дистанционный доступ счет',
             'ериб': 'единый розничный интернет банк',
             'нпф': 'негосударственный пенсионный фонд',
             'тсв': 'телевизионный система видеоконтроль',
             'оптинет': 'заявка перемещение денежный средства внутренний структурный подразделение',
             'optinet': 'заявка перемещение денежный средства внутренний структурный подразделение',
             'фир': 'файловый информационный ресурс',
             'ксш': 'корпоративный сервисный шина'}

In [ ]:
def filter(sent, stop=stop, abbr_dict=abbr_dict):
    sent = str(sent).split()
    filtered = []
    for s in sent:
        if abbr_dict.get(s):
            for word in abbr_dict.get(s).split():
                filtered.append(word)
        elif s not in stop:
            filtered.append(s)
    filtered = ' '.join([s for s in filtered])
    return filtered

In [ ]:
data['eng_norm'] = data['eng_norm'].apply(filter)
data.drop_duplicates(subset='eng_norm', inplace=True)
data['eng_norm'].replace('', np.nan, inplace=True)
data.dropna(subset=['eng_norm'], inplace=True)
scenarios = data['chosen_scenario']
sents = data['eng_norm'].to_list()
data = data.reset_index()
data = data.drop(columns=['index'])

# Создание эмбеддингов запросов

In [ ]:
# используем сжатую Fasttext модель, обученную на Национальном Корпусе Русского Языка

import fasttext
import fasttext.util
import compress_fasttext
small_model = compress_fasttext.models.CompressedFastTextKeyedVectors.load(
    'https://github.com/avidale/compress-fasttext/releases/download/v0.0.1/ft_freqprune_100K_20K_pq_100.bin'
)

In [ ]:
# Словарь частоты употребления каждого слова
def frequency(sents=sents):
    word_dict = {}
    for sent in sents:
        for word in sent.split():
            if word_dict.get(word):
                word_dict.update({word: word_dict.get(word) + 1})
            else:
                word_dict.update({word: 1})
    return word_dict

word_dict = frequency()

In [ ]:
# Из каждого запроса делается один вектор.
# Если слово одно, то просто преобразуем его в вектор, если их несколько, то используем most_similar метод
embeddings = []
for sent in sents:
    if len(sent) > 1:
        embeddings.append([small_model.most_similar(positive=sent)[0][0],])
    else:
        embeddings.append(sent)

# Загоняем векторные представления всех запросов в массив
vectors = np.zeros((len(embeddings),len(small_model[embeddings[0][0]])))
i = 0
for word in embeddings:
    vectors[i] = small_model[word]
    i += 1

In [ ]:
#пример использования метода most_similar

small_model.most_similar(positive=['король', 'женщина'], negative=['мужчина'])

[('королева', 0.6860082745552063),
 ('королевич', 0.6404726505279541),
 ('королевство', 0.6111368536949158),
 ('герцог', 0.5870347023010254),
 ('герцогиня', 0.5689640045166016),
 ('принцесса', 0.5578069090843201),
 ('принц', 0.5537070631980896),
 ('елисавета', 0.5434831380844116),
 ('королев', 0.5221450328826904),
 ('царица', 0.5178899168968201)]

# DBSCAN

In [ ]:
# Кластеризация при помощи DBSCAN алгоритма

import sklearn
print(sklearn.__version__)
from scipy.special import comb, logsumexp
from sklearn.cluster import DBSCAN
from sklearn import metrics 

0.23.2


In [ ]:
db = DBSCAN(eps=0.5, min_samples=300, algorithm='ball_tree').fit(vectors)

In [ ]:
db2 = db

In [ ]:
max(db.labels_)

22

In [ ]:
db.labels_


array([ 0,  0,  1, ..., 13, -1, 18])

# K-means

In [ ]:
# Кластеризация при помощи алгоритма K-means

from sklearn.cluster import KMeans 
from sklearn import metrics 
from scipy.spatial.distance import cdist 

In [ ]:
# Elbow method, с помощью которого можно выбрать оптимальное кол-во кластеров.
# Просто смотрим, когда точность модели по заданной метрике уменьшается уже не так сильно, и выбираем граничную точку
model = KMeans()
visualizer = KElbowVisualizer(model, k=(5,30), metric='calinski_harabaz')
visualizer.fit(vectors)        # Fit the data to the visualizer
visualizer.show()

In [ ]:
# Oбучение на полученном через elbow method значении
kmeans = KMeans(n_clusters=7, random_state=0).fit(vectors)

# Оценка результатов

In [ ]:
# Оценить результат можно на части размеченных данных

scenario = scenarios.drop_duplicates()
scenario = scenario.to_list()
scenario.remove('can_not_answer')
scenarios = scenarios.to_list()

In [ ]:
len(scenario)

465

In [ ]:
labeled_data = data.copy()
labeled_data['label'] = pd.Series(db.labels_)

In [ ]:
labeled_data.to_excel("labeled_data.xlsx")

In [ ]:
labeled_data

In [ ]:
def metric(labels, scenarios=scenarios, scenario=scenario):
    true = 0
    false = 0
    for scen_ex in scenario:
        scen_labels = {}
        k = 0
        for scen, label in zip(scenarios, labels):
            if scen_ex == scen:
                k += 1
                if scen_labels.get(label):
                    scen_labels.update({label: scen_labels.get(label) + 1})
                else:
                    scen_labels.update({label: 1})
        if k:
            all_values = scen_labels.values()
            true += max(all_values)
            false += k - max(all_values)
    print("true: " + str(true))
    print("false: " + str(false))
    return(true / (true + false))

def word_counter(labels, sents=sents):
    labels_dict = {}
    for label, sent in zip(labels, sents):
        if not labels_dict.get(label):
            labels_dict.update({label: {}})
        for word in sent.split():
            if labels_dict.get(label).get(word):
                labels_dict.get(label).update({word: labels_dict.get(label).get(word) + 1})
            else:
                labels_dict.get(label).update({word: 1})
    return(labels_dict)

def get_n_most_frequent(dictionary, n=10):
    dict_n = {}
    i = 0
    for k, v in sorted(dictionary.items(), key=lambda item: item[1], reverse=True):
        dict_n.update({k: v})
        if i < n:
            i += 1
        else:
            break
    return dict_n

def apply_to_all(dictionary, f):
    labels_dict_n = {}
    for k, v in dictionary.items():
        labels_dict_n.update({k: f(v)})
    return labels_dict_n

In [ ]:
m = metric(db.labels_)
labels_dict = word_counter(db.labels_)
labels_dict_n = apply_to_all(labels_dict, get_n_most_frequent)

true: 8913
false: 2881


In [ ]:
m

0.7557232491097168

In [ ]:
def print_n_random_requests(data, n=10):
    labels = data['label'].drop_duplicates().tolist()
    for label in labels:
        ld = data[data['label']==label]
        print(label)
        print(str(ld.sample(n)['normalized_text']))

print_n_random_requests(labeled_data, n=5)

In [ ]:
vector_df = pd.DataFrame(vectors)
vector_df.to_csv('vectors.csv', index=True)